# Resolution par methode directe de Gauss et factorisation LU


**Concepts abordés:**
* solution par echolonnement de gauss.
* decomposition $LU$ a l'aide de la methode de gauss.
* decomposition $LL^{T}$ a l'aide de la methode Cholskey.\

On a le systeme $Ax=b$ definie par A = $\begin{bmatrix} 4& 4&  2& 0\\ 4&  5&  0& 0\\ 2&  0&  6& 1\\ 0&  0&  1& 2 \end{bmatrix}$
et b $\in \mathbb{(R^4_{\ast})}$\
<br>
A l'aide de la bibliothèque NumPy qui sert à manipuler des matrices ou tableaux multidimensionnels ainsi que des fonctions mathématiques opérant sur ces tableaux.\
<br>
Onimporte un fichier "utils" contenons les fonctions utilises dans le code:
<br>
<br>
    * **matprint** : Rendre l' affichage des matrices plus agreable et lisible.
<br>
    * **GaussSeidel** : La fonction pour la resolution du system par methode de Gauss.
<br>
    * **Cholesky** : La fonction pour la resoulution du system par methode de Cholesky.
<br>
<br>
On a la matrice suivante :
<br>
<br>

$\begin{bmatrix} 4& 4&  2& 0\\ 4&  5&  0& 0\\ 2&  0&  6& 1\\ 0&  0&  1& 2 \end{bmatrix} \begin{bmatrix} x_{1} \\ x_{2} \\ x_{3} \\ x_{4} \end{bmatrix} = \begin{bmatrix} 1 \\ 2 \\3 \\ 4 \end{bmatrix}$

Commencons par l'integration du bibliotheque utilise  et les declaration initial :\

In [25]:
import numpy as np
import sys
from utils import solver

In [26]:
def matprint(mat, fmt="g"):
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    print("-"*100)
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
    print("-"*100)


On commence par definire notre matrice $A$ et $b$ dans python et les stacker pour creer une matrice 
augmenter qui va nous aider dans les calcules.

In [27]:
A = np.array([[4, 4, 2, 0], [4, 5, 0, 0], [2, 0, 6, 1], [0, 0, 1, 2]])
b = np.array([[1,2,3,4]],dtype='f').T
Aug_A_b = np.column_stack((A,b))

In [28]:
print("La matrice A :")
matprint(A)
print("La vecteur b.T")
matprint(b)
print("La matrice augmente est :")
matprint(Aug_A_b)

La matrice A :
----------------------------------------------------------------------------------------------------
4  4  2  0  
4  5  0  0  
2  0  6  1  
0  0  1  2  
----------------------------------------------------------------------------------------------------
La vecteur b.T
----------------------------------------------------------------------------------------------------
1  
2  
3  
4  
----------------------------------------------------------------------------------------------------
La matrice augmente est :
----------------------------------------------------------------------------------------------------
4  4  2  0  1  
4  5  0  0  2  
2  0  6  1  3  
0  0  1  2  4  
----------------------------------------------------------------------------------------------------


---
## Methode de Gauss :
- ### Matrice triangulaire superieur :
Avant de passer a l'algorithme rappelons d'abord le principe de la methode de gauss :
<br>
<br>
![](assets\1.png)

D'ou la methode de gauss consiste a passer par plusieur matrice de passage qui sert a transformer le systeme initiale et le changer a un systeme triangulaire superieur a l'aide des operations elementaire, et on peut l'implementer dans python a l'aide de l'algorithme suivant:
- ### Algorithme de la methode de Gauss :

![](assets/5.png)

- ### implementation sous Python :

In [29]:

def GaussSeidel(A,b):
    w = list()
    Aug_A = np.column_stack((A,b))
    n = np.shape(A)[0]
    x = np.array([np.zeros(n)],float).T
    
    for i in range(n):
        if Aug_A[i][i] == 0.0:
            sys.exit('Divide by zero detected!')
   
        for j in range(i+1, n):
            ratio = Aug_A[j][i]/Aug_A[i][i]
            
            for k in range(n+1):
                Aug_A[j][k] = Aug_A[j][k] - ratio * Aug_A[i][k]
            print(f"l{j} -> l{j} - {ratio} * l{i}")
            matprint(Aug_A)
            L = np.identity(4)
            L[j][i] = - ratio
            w.append(L) 
            
    x[n-1] = Aug_A[n-1][n]/Aug_A[n-1][n-1]
    
    for i in range(n-2,-1,-1):
        x[i] = Aug_A[i][n]
     
        for j in range(i+1,n):
            x[i] = x[i] - Aug_A[i][j]*x[j]

        x[i] = x[i]/Aug_A[i][i]

    L = np.linalg.multi_dot([w[5],w[4],w[3],w[2],w[1],w[0]])
    L = np.linalg.inv(L)
    U = np.linalg.multi_dot([w[5],w[4],w[3],w[2],w[1],w[0],A])
    return x,w,L,U

Voyons le resultats de la methode par Gauss avec les etapes de l'echelonnemt :

In [30]:
x1,w,L,U= GaussSeidel(A,b)
print("La solution par methode de Gauss")
matprint(x1.T)

l1 -> l1 - 1.0 * l0
----------------------------------------------------------------------------------------------------
4  4   2  0  1  
0  1  -2  0  1  
2  0   6  1  3  
0  0   1  2  4  
----------------------------------------------------------------------------------------------------
l2 -> l2 - 0.5 * l0
----------------------------------------------------------------------------------------------------
4   4   2  0    1  
0   1  -2  0    1  
0  -2   5  1  2.5  
0   0   1  2    4  
----------------------------------------------------------------------------------------------------
l3 -> l3 - 0.0 * l0
----------------------------------------------------------------------------------------------------
4   4   2  0    1  
0   1  -2  0    1  
0  -2   5  1  2.5  
0   0   1  2    4  
----------------------------------------------------------------------------------------------------
l2 -> l2 - -2.0 * l1
-------------------------------------------------------------------------------------

la solution finale est :
&nbsp;&nbsp; $x = \begin{bmatrix} -13.25 \\ 11 \\ 5 \\ 0.5 \end{bmatrix}$

- ### Decomposition $LU$ Par methode de Gauss:

On peut obtenir la decomposition $LU$ a l'aide de methode de Gauss d'ou le produit des matrice de permutation par la matrice $A$ nous donne la matrice $U$ :\
Par exemple :
pour l'operation &nbsp;&nbsp; $l_{1} -> l_{1} - 1.0 \times l{_0}$\
<br>
$w_{0} = \begin{bmatrix}1& 0& 0& 0& \\ -1& 1& 0& 0& \\ 0& 0& 1& 0& \\ 0& 0& 0& 1& \end{bmatrix}$\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\vdots$
<br>\
d'ou on proced de meme facons jusqua quand obtenue notre matrice $U$ a l'aide de Gauss, et pour lamtrice $L$ on peut l'avoir par :

$W = w_{0}w_{1}w_{2} \cdots w_{n}$

avec : $WA = U$
et 
$WA = U \Leftrightarrow A = W^{-1}U$ on posons $W^{-1} = L$\
On peut voir tout les matrice de permutation par

In [31]:
for i in range(6):
    print(f"la matrice w[{i}]")
    matprint(w[i])

la matrice w[0]
----------------------------------------------------------------------------------------------------
 1  0  0  0  
-1  1  0  0  
 0  0  1  0  
 0  0  0  1  
----------------------------------------------------------------------------------------------------
la matrice w[1]
----------------------------------------------------------------------------------------------------
   1  0  0  0  
   0  1  0  0  
-0.5  0  1  0  
   0  0  0  1  
----------------------------------------------------------------------------------------------------
la matrice w[2]
----------------------------------------------------------------------------------------------------
 1  0  0  0  
 0  1  0  0  
 0  0  1  0  
-0  0  0  1  
----------------------------------------------------------------------------------------------------
la matrice w[3]
----------------------------------------------------------------------------------------------------
1  0  0  0  
0  1  0  0  
0  2  1  0  
0  0  0  1  
-

Donc on aurra la decomposition de $LU$ par la forme :

In [32]:
print("La decomposition U methode de Gauss")
matprint(U)
print("La decomposition L methode de Gauss")
matprint(L)   
print("solution LU de methode de Gauss")
x2 = solver(L,U,b)
matprint(x2)
    

La decomposition U methode de Gauss
----------------------------------------------------------------------------------------------------
4  4   2  0  
0  1  -2  0  
0  0   1  1  
0  0   0  1  
----------------------------------------------------------------------------------------------------
La decomposition L methode de Gauss
----------------------------------------------------------------------------------------------------
  1  -0  -0  -0  
  1   1   0   0  
0.5  -2   1  -0  
  0   0   1   1  
----------------------------------------------------------------------------------------------------
solution LU de methode de Gauss
----------------------------------------------------------------------------------------------------
-13.25  11  5  -0.5  
----------------------------------------------------------------------------------------------------


---


## Methode de Choleskey
On cherche la matrice :
$L=\begin{bmatrix}
l_{11}\\
l_{21} & l_{22}\\
\vdots & \vdots & \ddots\\
l_{n1} & l_{n2} & \cdots & l_{nn}
\end{bmatrix}$

De l'égalité $A = LL^{T}$ on déduit :

$a_{ij}=\left(LL^{T}\right)_{ij}={\sum_{k=1}^{n}l_{ik}l_{jk}}={\sum_{k=1}^{\min\left\{ i,j\right\} }l_{ik}l_{jk}},\;1\leq i,j\leq n$

puisque $l_{pq}=0$ si $1 ≤ p < q ≤ n$.

La matrice $A$ étant symétrique, il suffit que les relations ci-dessus soient vérifiées pour $i ≤ j$, c'est-à-dire que les éléments $l_{ij}$ de la matrice $L$ doivent satisfaire :

$a_{ij}={\sum_{k=1}^{i}l_{ik}l_{jk}},\;1\leq i\leq j\leq n$

Pour $i=1$, on détermine la première colonne de $L$ :

$a_{11}=l_{11}l_{11}$ d'où $l_{11}=a_{11}$
$a_{1j}=l_{11}l_{j1}$ d'où $l_{j1}=a_{1j}l_{11}$ ,$2≤j≤$

On détermine la i-ème colonne de $L 2 ≤ i ≤ n$, après avoir calculé les (i–1) premières colonnes :

$a_{ii}=l_{i1}l_{i1}+\ldots+l_{ii}l_{ii}$ d'où $l_{ii}= \sqrt{{a_{ii}-{\sum_{k=1}^{i-1}l_{ik}^{2}}}}$

$a_{ij}=l_{i1}l_{j1}+\ldots+l_{ii}l_{ji}$ d'où $l_{ji}=\frac{a_{ij}-{\sum_{k=1}^{i-1}l_{ik}l_{jk}}}{l_{ii}},\;\;\;i+1\leq j\leq n$

Il résulte du théorème précédent qu'il est possible de choisir tous les éléments $l_{ii}>0$ en assurant que toutes les quantités

$a_{11},\ldots,a_{ii}-{\sum_{k=1}^{i-1}l_{ik}^{2}},\ldots$

sont positives.

![](assets/6.png)

- ### Algorithme Cholesky :

In [33]:
def cholesky(A):
    L = np.zeros_like(A, float)
    n = len(L)
    for i in range(n):
        for j in range(i+1):
            if i==j:
                val = A[i,i] - np.sum(np.square(L[i,:i]))
                # if diagonal values are negative return zero - not throw exception
                if val<0:
                    return 0.0
                L[i,i] = np.sqrt(val)
            else:
                L[i,j] = (A[i,j] - np.sum(L[i,:j]*L[j,:j]))/L[j,j]
                
    return L

La fonction `Cholesky()` prend la matrice $A$ comme argument et retourne la matrice $L$, et avec l'aide 
de la fonction  `solver()`qui prend $U$,$L$ et $b$ comme argument et retourne les valuers du vecteur $x$. 

In [34]:
def solver(L,U,b):
  L=np.array(L, float)
  U=np.array(U, float)
  b=np.array(b, float)

  n,_=np.shape(L)
  y=np.zeros(n)
  x=np.zeros(n)

# Forward substitution
  for i in range(n):
    sumj=0
    for j in range(i):
      sumj += L[i,j]*y[j]
    y[i]=(b[i]-sumj)/L[i,i]

# Backward substitution  
  for i in range(n-1, -1, -1):
    sumj=0
    for j in range(i+1,n):
      sumj += U[i,j] * x[j]
    x[i]=(y[i]-sumj)/U[i,i]
  
  return np.array([x])

In [35]:
L = cholesky(A)
print('La matrice triangulaire inferieur L :')
matprint(L)
U = np.transpose(L)
print('La matrice triangulaire superieur U :')
matprint(U)
print("La solution par methode de Cholesky")
x2 = solver(L, U, b)
matprint(x2)

La matrice triangulaire inferieur L :
----------------------------------------------------------------------------------------------------
2   0  0  0  
2   1  0  0  
1  -2  1  0  
0   0  1  1  
----------------------------------------------------------------------------------------------------
La matrice triangulaire superieur U :
----------------------------------------------------------------------------------------------------
2  2   1  0  
0  1  -2  0  
0  0   1  1  
0  0   0  1  
----------------------------------------------------------------------------------------------------
La solution par methode de Cholesky
----------------------------------------------------------------------------------------------------
-13.25  11  5  -0.5  
----------------------------------------------------------------------------------------------------


la solution finale est :
&nbsp;&nbsp; $x = \begin{bmatrix} -13.25 \\ 11 \\ 5 \\ 0.5 \end{bmatrix}$\
D'ou on constate qu'on eu le meme resultat par deux methode differente.

---
## Matrice symetrique definie positife :
![](assets/4.png)

* ### Par la methode des valeurs propre :

In [36]:
def is_pos_def(A):
    if np.all(np.linalg.eigvals(A) > 0):
        print('la matrice A est superieur definie positive.')
        return True
    else:
        print('la matrice A n\'est pas superieur definie positive.')
        return False

is_pos_def(A)

la matrice A est superieur definie positive.


True

* ### Par les sous Matrice

In [37]:
print("-"*100,"\nLes sous matrice A :\n","-"*100)
a1 = A[0:1,0:1]
matprint(a1)
print(f"sont det(a1) = {np.linalg.det(a1):0.4}")
print("-"*100,"\nLes sous matrice A :\n","-"*100)
a2 = A[0:2,0:2]
matprint(a2)
print(f"sont det(a22) = {np.linalg.det(a2):0.4} ")
print("-"*100,"\nLes sous matrice A :\n","-"*100)
a3 = A[0:3,0:3]
matprint(a3)
print(f"sont det(a3) = {np.linalg.det(a3):0.4} ")
print("-"*100,"\nLes sous matrice A :\n","-"*100)
a4 = A[0:4,0:4]
matprint(a4)
print(f"sont det(a4) = {np.linalg.det(a4):0.4} ")

---------------------------------------------------------------------------------------------------- 
Les sous matrice A :
 ----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
4  
----------------------------------------------------------------------------------------------------
sont det(a1) = 4.0
---------------------------------------------------------------------------------------------------- 
Les sous matrice A :
 ----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
4  4  
4  5  
----------------------------------------------------------------------------------------------------
sont det(a22) = 4.0 
----------------------------------------------------------------------------------------